In [2]:
import numpy as np
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data
import os


from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

import acquire
import prepare

In [3]:
def prep_telco():
    telco_churn_db = acquire.get_telco_data()
    telco_churn_db = telco_churn_db.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id'])
    dummy_telco_churn_db = pd.get_dummies(telco_churn_db[['gender', 'partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing', 'churn', 'contract_type', 'internet_service_type', 'payment_type']], dummy_na=False, drop_first=True)
    telco_churn_db = pd.concat([telco_churn_db, dummy_telco_churn_db], axis=1)
    return telco_churn_db

In [4]:
t = acquire.get_telco_data()
t.shape

(7043, 25)

In [5]:
def prep_telco_alternative():
    telco_churn_db = acquire.get_telco_data()
    telco_churn_db = telco_churn_db.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id'])
    dummy_telco_churn_db = pd.get_dummies(telco_churn_db[['gender', 'partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing', 'churn', 'internet_service_type', 'payment_type']], dummy_na=False, drop_first=True)
    #Since month-to-month seems VERY correlated to churn rate I am not dropping first when doing contract type specifically for knn nearest neighbors where I have to cut down the dimensions and choose the best correlated issues.  
    #I expect the decision trees to give me more intuition on the most highly correlated dimensions to choose
    dummy_telco_churn_db_first = pd.get_dummies(telco_churn_db[['contract_type']], dummy_na=False, drop_first=False) 
    telco_churn_db = pd.concat([telco_churn_db, dummy_telco_churn_db, dummy_telco_churn_db_first], axis=1)
    return telco_churn_db

In [6]:
def split_data(df, stratify_col):
    train_validate, test = train_test_split(df, test_size = .2, random_state=823, stratify=df[stratify_col])
    train, validate = train_test_split(train_validate, test_size=.25, random_state=823, stratify=train_validate[stratify_col])
    return train, validate, test

In [7]:
telco = prep_telco_alternative()
telco.dtypes

Unnamed: 0                                 int64
customer_id                               object
gender                                    object
senior_citizen                             int64
partner                                   object
dependents                                object
tenure                                     int64
phone_service                             object
multiple_lines                            object
online_security                           object
online_backup                             object
device_protection                         object
tech_support                              object
streaming_tv                              object
streaming_movies                          object
paperless_billing                         object
monthly_charges                          float64
total_charges                             object
churn                                     object
contract_type                             object
internet_service_typ

In [8]:
telco = telco.drop(columns=telco.select_dtypes(exclude='number').columns)

In [9]:
telco.dtypes

Unnamed: 0                                 int64
senior_citizen                             int64
tenure                                     int64
monthly_charges                          float64
gender_Male                                uint8
partner_Yes                                uint8
dependents_Yes                             uint8
phone_service_Yes                          uint8
multiple_lines_No phone service            uint8
multiple_lines_Yes                         uint8
online_security_No internet service        uint8
online_security_Yes                        uint8
online_backup_No internet service          uint8
online_backup_Yes                          uint8
device_protection_No internet service      uint8
device_protection_Yes                      uint8
tech_support_No internet service           uint8
tech_support_Yes                           uint8
streaming_tv_No internet service           uint8
streaming_tv_Yes                           uint8
streaming_movies_No 

In [10]:
telco.churn_Yes.value_counts()

0    5174
1    1869
Name: churn_Yes, dtype: int64

In [11]:
baseline = 1869/(1869+5174)
baseline

0.2653698707936959

Generalized Null hypothesis (For each Column): "Column" is independant of the Churn rate
Generalized Alt hypothesis (For each Column): "Column" and Churn rate are dependant
\alpha < 0.01.  Higher alpha is justified because n is large and there are so many columns to choose from

In [12]:
for col in telco.columns:
    col = pd.crosstab(telco[col], telco.churn_Yes)
    print(col)
    col_chi2, col_p, col_degf, col_expected = stats.chi2_contingency(col)
    print(f'(chi squared = {col_chi2}, p = {col_p}')
    print('___________________________________________________________')

churn_Yes   0  1
Unnamed: 0      
0           1  0
1           1  0
2           0  1
3           0  1
4           0  1
...        .. ..
7038        1  0
7039        0  1
7040        1  0
7041        1  0
7042        1  0

[7043 rows x 2 columns]
(chi squared = 7043.0, p = 0.49439767459438705
___________________________________________________________
churn_Yes          0     1
senior_citizen            
0               4508  1393
1                666   476
(chi squared = 159.42630036838742, p = 1.510066805092378e-36
___________________________________________________________
churn_Yes    0    1
tenure             
0           11    0
1          233  380
2          115  123
3          106   94
4           93   83
5           69   64
6           70   40
7           80   51
8           81   42
9           73   46
10          71   45
11          68   31
12          79   38
13          71   38
14          52   24
15          62   37
16          52   28
17          61   26
18          73   2

"Unnamed: 0", "gender_male", "phone_service_Yes", "multiple_lines_No phone service" all do not meet the p-value we set and thus we do not have sufficient evidence to reject the null hypothesis.  Therefore I am dropping them all from the analysis.  "Unnamed: 0" is just an id number so it makes sense that they are independant events...kinda goes without saying but a good sanity check.

In [13]:
telco = telco.drop(columns=['Unnamed: 0', 'gender_Male', 'phone_service_Yes', 'multiple_lines_No phone service'])
telco

,senior_citizen,tenure,monthly_charges,partner_Yes,dependents_Yes,multiple_lines_Yes,online_security_No internet service,online_security_Yes,online_backup_No internet service,online_backup_Yes,...,paperless_billing_Yes,churn_Yes,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check,contract_type_Month-to-month,contract_type_One year,contract_type_Two year
0,0,9,65.60,1,1,0,0,0,0,1,...,1,0,0,0,0,0,1,0,1,0
1,0,9,59.90,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2,0,4,73.90,0,0,0,0,0,0,0,...,1,1,1,0,0,1,0,1,0,0
3,1,13,98.00,1,0,0,0,0,0,1,...,1,1,1,0,0,1,0,1,0,0
4,1,3,83.90,1,0,0,0,0,0,0,...,1,1,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,13,55.15,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
7039,0,22,85.10,1,0,1,0,0,0,0,...,1,1,1,0,0,1,0,1,0,0
7040,0,2,50.30,0,0,0,0,0,0,1,...,1,0,0,0,0,0,1,1,0,0
7041,0,67,67.85,1,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
